In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
communities = pd.read_csv("../Data/communities.csv")
egm = pd.read_csv("../Data/EGM.csv")
housingPrice = pd.read_csv("../Data/Houses-by-suburb.csv")
offences = pd.read_csv("../Data/LGA_Offences_Table1.csv")

In [3]:
def clean_lga(lga: str):
    lga = lga.strip().lower()
    lga = lga.replace('-', '')
    replaces = {
        'moreland': 'merribek'
    }
    lga = ''.join([replaces.get(word, word) for word in lga.split() if word not in ['of', 'city', 'rural', 'shire', 'borough']])
    # Remove the bracket from communities
    lga = lga.split('(')[0]
    return lga


In [4]:
# Clean the LGA of the 3 tables (Housing price is special)
communities["LGA"] = communities["LGA"].apply(clean_lga)
egm["LGA Name"] = egm["LGA Name"].apply(clean_lga)
offences["Local Government Area"] = offences["Local Government Area"].apply(clean_lga)

In [5]:
# # Try to scrape the government website
# # Get the list of 79 LGA (local councils)
# def getLGAFromCouncilLink(url: str):
#     rePattern = r'/know-your-council-(.*)'
#     councilName = re.match(rePattern, url).group(1)
#     councilName = councilName.strip().lower()
#     words = councilName.split('-')
#     lga = ''.join([word for word in words if word not in ['of', 'city', 'rural', 'shire', 'borough', 'council']])
#     return lga

# prefixUrl = "https://www.vic.gov.au"
# page = requests.get("https://www.vic.gov.au/know-your-council")
# soup = BeautifulSoup(page.text, 'html.parser')

# councilList = soup.find("ul", {"class": 'vic-kyc__councils-list rpl-grid'})
# allLinksToCouncil = councilList.find_all("a")
# lgaLocality = {}

# for anchorTag in allLinksToCouncil:
#     link = anchorTag["href"]
#     lga = getLGAFromCouncilLink(link)

#     page = requests.get(prefixUrl + link)
#     localitySoup = BeautifulSoup(page.text, 'html.parser')
#     localityHeader = localitySoup.find(id="localities")
#     localityParagraph = localityHeader.find_next("p")
#     localityText = localityParagraph.text.strip(" .").replace(" and ", ", ")
#     locailityList = localityText.split(", ")

#     lgaLocality[lga] = locailityList

# # with open("lgaLocality.json", "w") as outFile:
# #     json.dump(lgaLocality, outFile, indent=4)

# lgaLocality


In [6]:
# # Try using the scraped data to map locality to LGA
# # There are missing localities, so mainly use this to find the missing localities
# with open("../lgaLocality.json", "r") as file:
#     lgaLocality = json.load(file)

# def clean_communities(community: str):
#     community = community.strip().lower()
#     # Strip off any brackets at the end
#     community = community.split("(")[0]
#     # Rip off directions
#     words = list(community.split())
#     for i in range(len(words)):
#         if words[i] in ("north", "south", "east", "west", "central"):
#             words[i] = ""
#     return ''.join(words)


# housingPrice["Locality"] = housingPrice["Locality"].apply(clean_communities)

# mapDictionary = {}
# cannotMap = []

# for housingPriceCommunity in housingPrice["Locality"]:
#     mapped = False
#     for lga in lgaLocality:
#         for community in lgaLocality[lga]:
#             community = clean_communities(community)
#             if housingPriceCommunity in community:
#                 mapDictionary[housingPriceCommunity] = (community, lga)
#                 mapped = True
#                 break
#         if mapped:
#             break
#     if not mapped:
#         cannotMap.append(housingPriceCommunity)

# print(mapDictionary)
# print(cannotMap)
# print(mapDictionary['osborne'])


In [7]:
# # Compute the full list to convert community to LGA
# def clean_communities(community: str):
#     community = community.strip().lower()
#     # Strip off any brackets at the end
#     community = community.split("(")[0]
#     # Rip off directions
#     words = list(community.split())
#     for i in range(len(words)):
#         if words[i] in ("north", "south", "east", "west", "central"):
#             words[i] = ""
#     return ''.join(words)

# with open("../lgaLocality.json", "r") as file:
#     lgaLocality = json.load(file)

# handMapLocalityToLGA = {'aintree': 'melton', 
#                       'balcombe': 'wodonga', 
#                       'benalla': 'benalla', 
#                       'bonniebrook': 'melton', 
#                       'botanicridge': 'casey', 
#                       'capelsound': 'morningtonpeninsula', 
#                       'cobblebank': 'melton', 
#                       'coonanshill': 'merribek', 
#                       'danyo': 'mildura', 
#                       'deanside': 'melton', 
#                       'deepdene': 'boroondara', 
#                       'eastwood': 'eastgippsland', 
#                       'fiveways': 'latrobe', 
#                       'fraserrise': 'melton', 
#                       'glenwaverley': 'monash', 
#                       'harkness': 'melton', 
#                       'jolimont': 'greatershepparton', 
#                       'laralake': 'greatergeelong', 
#                       'lucas': 'ballarat', 
#                       'manorlakes': 'wyndham', 
#                       'mansfield': 'mansfield', 
#                       'mckenziehill': 'mountalexander', 
#                       'merindapark': 'casey', 
#                       'pattersongardens': 'brimbank', 
#                       'sanctuarylakes': 'wyndham', 
#                       'sthelena': 'banyule', 
#                       'strathtulloh': 'melton', 
#                       'thornhillpark': 'melton', 
#                       'weirviews': 'melton', 
#                       'westall': 'greaterdandenong', 
#                       'westgarth': 'nillumbik', 
#                       'wimbledonheights': 'basscoast', 
#                       'wintervalley': 'ballarat', 
#                       'woolamaiwaters': 'basscoast'}
# localityToLGA = {}
# for community in mapDictionary:
#     localityToLGA[community] = mapDictionary[community][1]

# for community in handMapLocalityToLGA:
#     localityToLGA[community] = handMapLocalityToLGA[community]
# with open("../localityToLGA.json", "w") as outFile:
#     json.dump(localityToLGA, outFile, indent=4)

# print(len(localityToLGA))

In [9]:
def processHousingPriceLGA(housingPrice):
    with open("../localityToLGA.json", "r") as file:
        localityToLGAMap = json.load(file)


    def clean_communities(community: str):
        community = community.strip().lower()
        # Strip off any brackets at the end
        community = community.split("(")[0]
        # Rip off directions
        words = list(community.split())
        for i in range(len(words)):
            if words[i] in ("north", "south", "east", "west", "central"):
                words[i] = ""
        return ''.join(words)

    def localityToLGA(locality: str, localityToLGAMap: dict):
        return localityToLGAMap[clean_communities(locality)]


    housingPrice["LGA"] = housingPrice["Locality"].apply(lambda x: localityToLGA(x, localityToLGAMap))

processHousingPriceLGA(housingPrice)
housingPrice["LGA"]

0             yarra
1      mooneevalley
2            melton
3         surfcoast
4              hume
           ...     
782       nillumbik
783     maribyrnong
784           moira
785     murrindindi
786         latrobe
Name: LGA, Length: 787, dtype: object

In [10]:
allLGAs = {
    "communities": communities["LGA"].unique(),
    "egm": egm["LGA Name"].unique(),
    "housingPrice": housingPrice["LGA"].unique(),
    "offences": offences["Local Government Area"].unique()
}

allLGAs

{'communities': array(['yarra', 'mooneevalley', 'surfcoast', 'brimbank', 'portphillip',
        'murrindindi', 'pyrenees', 'ballarat', 'warrnambool', 'darebin',
        'hobsonsbay', 'colacotway', 'ararat', 'greatershepparton',
        'stonnington', 'morningtonpeninsula', 'greaterbendigo',
        'boroondara', 'monash', 'kingston', 'hume', 'strathbogie',
        'cardinia', 'moorabool', 'eastgippsland', 'southerngrampians',
        'greaterdandenong', 'goldenplains', 'wodonga', 'indigo',
        'greatergeelong', 'frankston', 'maroondah', 'knox', 'bayside',
        'yarraranges', 'towong', 'banyule', 'benalla', 'gleneira', 'casey',
        'yarriambiack', 'buloke', 'whitehorse', 'latrobe', 'loddon',
        'maribyrnong', 'wellington', 'alpine', 'mitchell', 'merribek',
        'manningham', 'bawbaw', 'moira', 'melton', 'mildura',
        'corangamite', 'basscoast', 'centralgoldfields', 'melbourne',
        'glenelg', 'mountalexander', 'hepburn', 'gannawarra', 'wyndham',
        'nill

In [13]:
commonLGAs = set(allLGAs["communities"]) \
    .intersection(set(allLGAs["egm"]))\
    .intersection(set(allLGAs["housingPrice"]))\
    .intersection(set(allLGAs["offences"]))

print(len(commonLGAs))
commonLGAs

56


{'alpine',
 'ballarat',
 'banyule',
 'basscoast',
 'bawbaw',
 'bayside',
 'benalla',
 'boroondara',
 'brimbank',
 'campaspe',
 'cardinia',
 'casey',
 'centralgoldfields',
 'colacotway',
 'darebin',
 'eastgippsland',
 'frankston',
 'gleneira',
 'glenelg',
 'greaterbendigo',
 'greaterdandenong',
 'greatergeelong',
 'greatershepparton',
 'hobsonsbay',
 'horsham',
 'hume',
 'kingston',
 'knox',
 'latrobe',
 'macedonranges',
 'manningham',
 'maribyrnong',
 'maroondah',
 'melbourne',
 'melton',
 'merribek',
 'mildura',
 'mitchell',
 'monash',
 'mooneevalley',
 'moorabool',
 'morningtonpeninsula',
 'northerngrampians',
 'portphillip',
 'southgippsland',
 'stonnington',
 'surfcoast',
 'swanhill',
 'wangaratta',
 'wellington',
 'whitehorse',
 'whittlesea',
 'wodonga',
 'wyndham',
 'yarra',
 'yarraranges'}